[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/pantat88/segsmaker/blob/main/Segsmaker_ComfyUI.ipynb)

## Conda

In [ ]:
!curl -sLo ~/.conda/conda_install.py https://github.com/gutris1/segsmaker/raw/main/script/conda3.py
%run ~/.conda/conda_install.py

## Clone ComfyUI

In [ ]:
%say 【 {red} Installing ComfyUI {d} 】 {red}
%cd -q ~
!git clone -q https://github.com/comfyanonymous/ComfyUI

%cd -q ~/ComfyUI
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl \
-r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118
%pull https://github.com/gutris1/segsmaker cui ~/ComfyUI
%download ~/ComfyUI/asd/cui.txt
!python ~/ComfyUI/asd/cui.py > /dev/null

''' Custom Nodes '''
%say 【 {red} Installing Custom Nodes {d} 】 {red}
%cd -q ~/ComfyUI/custom_nodes
%clone ~/ComfyUI/asd/custom_nodes.txt
%download ~/ComfyUI/asd/custom_nodes_models.txt
# add custom nodes here !git clone URL

%cd -q ~/ComfyUI
!pip install -q opencv-python==4.7.0.72 transformers==4.26.1 tokenizers clip-interrogator
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" > /dev/null
!rm -rf ~/.cache/* ~/ComfyUI/custom_nodes/~
%say 【 {red} Done {d} 】 {red}

## *ControlNET models* <span style="color:#ff7171; font-size: 14px;">temporary</span>

In [9]:
%tempe
%cd -q ~/ComfyUI/models/controlnet
%run ~/ComfyUI/asd/controlnet.py

Output()

## *Checkpoint* <span style="color:#ff7171; font-size: 14px;">temporary</span>

In [ ]:
%tempe
%cd -q ~/ComfyUI/models/checkpoints

%download https://civitai.com/api/download/models/287641 Meichidark_V5.safetensors

## *Lora* <span style="color:#ff7171; font-size: 14px;">temporary</span>

In [ ]:
%tempe
%cd -q ~/ComfyUI/models/loras

%download https://civitai.com/api/download/models/87153  add_detail.safetensors
%download https://civitai.com/api/download/models/171989 detail_slider_v4.safetensors
%download https://civitai.com/api/download/models/62833  Detail_Tweaker.safetensors
%download https://civitai.com/api/download/models/86247  flat2.safetensors

## Run ComfyUI

In [ ]:
''' run this cell once to install zrok and register an account '''
%cd -q ~
!python ~/ComfyUI/asd/zorok.py

In [ ]:
token = "YOUR_ZROK_TOKEN"

%cd -q ~/ComfyUI
!python zrok.py {token} & python main.py --dont-print-server

## Extras

#### ```zip``` Output images

In [ ]:
%%zipping

input_folder  = '/home/studio-lab-user/ComfyUI/output'
output_folder = '/home/studio-lab-user'

#### Storage

In [ ]:
''' Check Storage '''
!rm -rf ~/.cache/*
%storage /home/studio-lab-user

In [ ]:
''' Delete output images '''
!rm -rf ~/ComfyUI/output/* ~/.cache/*

In [ ]:
''' Delete ComfyUI '''
%cd -q ~
!rm -rf ~/tmp/* ~/ComfyUI ~/tmp ~/.cache/*

In [ ]:
''' DELETE EVERYTHING , restart jupyterlab afterward '''
%delete /home/studio-lab-user